In [ ]:
#Affiliation with AI Technology & Systems
#Comparing F1 score of brain tumor dataset

# building cnn
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from scipy import ndimage,misc
import os
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score
#initialising cnn
classifier = Sequential()

classifier.add(Convolution2D(32,3,3, input_shape =(64,64,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (3, 3)))

classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))

classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',f1])
train_path = 'train_data/'
test_path = 'test_data/'
#F1 score
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
                                                   train_path,
                                                   target_size=(64,64),
                                                   classes=['yes','no'],
                                                   batch_size= 32,
                                                   class_mode='binary')

test_set = test_datagen.flow_from_directory(
                                            test_path,
                                            target_size=(64,64),
                                            classes=['yes','no'],
                                            batch_size= 32,
                                            class_mode='binary')

#Training Model
classifier.fit_generator(
                         train_set,
                         samples_per_epoch=209,
                         epochs=25,
                         validation_data= test_set,
                         nb_val_samples=44)

#f1 score= 0.8319